In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Projects/TimeSeriesForecasting

/content/drive/My Drive/Projects/TimeSeriesForecasting


In [0]:
import os
import json
import numpy as np
import pandas as po
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [0]:
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import ConvLSTM2D, Dense, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
os.makedirs('model_checkpoints', exist_ok=True)
os.makedirs('plots', exist_ok=True)

In [0]:
df = po.read_csv('data/2017_energy_5min_noTransform.csv')

In [0]:
scaler = MinMaxScaler()
df['W'] = scaler.fit_transform(df)

In [0]:
class Args:
  def __init__(self):
    self.val_start = 12*24*250
    self.test_start = 12*24*300
    self.train_seq_len = 12*24*7
    self.val_seq_len = 12*24
    
    self.batch_size = 512
    self.num_epochs = 100
    self.save_folder = 'convLSTM'
    
    self.subsequences = self.train_seq_len//(12*24)
    self.conv_num_channels = 128
    self.kernel_size = (1, 12)

In [0]:
args = Args()

In [0]:
os.makedirs('plots/' + args.save_folder, exist_ok=True)
os.makedirs('model_checkpoints/' + args.save_folder, exist_ok=True)

In [0]:
train_df = df[: args.val_start]
val_df = df[args.val_start: args.test_start]
test_df = df[args.test_start:]

In [0]:
#scaler.inverse_transform(train)

In [0]:
def make_windows(data, train_seq_len = args.train_seq_len, val_seq_len = args.val_seq_len):
    X = []
    y = []

    for i in tqdm(range(train_seq_len, len(data) - val_seq_len + 1)):    
        X.append(data.values[i - train_seq_len: i].T.tolist())
        y.append(data.values[i: i + val_seq_len].T.tolist())

    X = np.array(X).squeeze().reshape(-1, train_seq_len, 1)
    y = np.array(y).squeeze()

    return X, y

In [0]:
X_train, y_train = make_windows(train_df)

100%|██████████| 69697/69697 [00:12<00:00, 5596.80it/s]


In [0]:
X_val, y_val = make_windows(val_df)

100%|██████████| 12097/12097 [00:01<00:00, 7874.12it/s]


In [0]:
X_train = X_train.reshape(X_train.shape[0], args.subsequences, 1, -1, 1)
X_train.shape

(69697, 7, 1, 288, 1)

In [0]:
X_val = X_val.reshape(X_val.shape[0], args.subsequences, 1, -1, 1)
X_val.shape

(12097, 7, 1, 288, 1)

In [0]:
model = Sequential()
model.add(ConvLSTM2D(filters = args.conv_num_channels, kernel_size = args.kernel_size, activation = 'relu', input_shape = (args.subsequences, 1, X_train.shape[3], 1)))
model.add(Flatten())
model.add(Dense(args.val_seq_len))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 1, 277, 128)       793088    
_________________________________________________________________
flatten_1 (Flatten)          (None, 35456)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 288)               10211616  
Total params: 11,004,704
Trainable params: 11,004,704
Non-trainable params: 0
_________________________________________________________________


In [0]:
sgd = optimizers.SGD(lr=0.1, decay=1e-2, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2)
best_val = ModelCheckpoint('model_checkpoints/' + args.save_folder + '/model_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)

In [0]:
history = model.fit(X_train, y_train, batch_size = args.batch_size, epochs = args.num_epochs, validation_data = (X_val, y_val), callbacks=[earlystop, best_val], verbose=1)

Train on 69697 samples, validate on 12097 samples
Epoch 1/100
57344/69697 [=======================>......] - ETA: 18s - loss: 0.0213

In [0]:
model.load_weights('model_checkpoints/' + args.save_folder + '/model_{:02d}.h5'.format(best_epoch))
best_epoch = np.argmin(np.array(history.history['val_loss']))+1
best_epoch

In [0]:
plot_df = po.DataFrame.from_dict({'train_loss':history.history['loss'], 'val_loss':history.history['val_loss']})
plot_df.plot(logy=True, figsize=(10,10), fontsize=12)
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.savefig('plots/'  + args.save_folder + '/train_val_loss.png')
plt.show()

In [0]:
X_test, y_test = make_windows(test_df)

In [0]:
X_test = X_test.reshape(X_test.shape[0], args.subsequences, 1, -1, 1)
X_test.shape

In [0]:
y_pred = model.predict(X_test)

In [0]:
plt.plot(y_test[0], color = 'blue')
plt.plot(y_pred[0], color = 'red')
plt.savefig('plots/' + args.save_folder + '/test_weekend_pred.png')

In [0]:
plt.plot(y_test[12*24*2], color = 'blue')
plt.plot(y_pred[12*24*2], color = 'red')
plt.savefig('plots/'  + args.save_folder + '/test_weekday_pred.png')

In [0]:
if not os.path.exists('results.csv'):
  results = po.DataFrame(columns = ['model', 'MAE', 'MSE', 'R2'])  
else:
  results = po.read_csv('results.csv')

In [0]:
errors = {'model': args.save_folder,
          'MAE': mean_absolute_error(y_true=y_test, y_pred=y_pred),
          'MSE': mean_squared_error(y_true=y_test, y_pred=y_pred),
          'R2': r2_score(y_true=y_test, y_pred=y_pred)
          }
errors = po.Series(errors)
results = results.append(errors, ignore_index=True)

In [0]:
results

In [0]:
results.to_csv('results.csv', index=False)

In [0]:
with open('model_checkpoints/' + args.save_folder + '/params.json', 'w') as f:
  f.write(json.dumps(args.__dict__))
f.close()